In [159]:
import tomllib
from pathlib import Path
from IPython.display import display
from decimal import Decimal

In [117]:
test_content = tomllib.load(Path(r"../tests/assets/test.toml").open('rb'))
display(test_content)

{'title': 'TOML Example',
 'owner': {'name': 'Tom Preston-Werner',
  'dob': datetime.datetime(1979, 5, 27, 7, 32, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600))),
  'nb_of_days': 34,
  'calculated_value': '= nb_of_days * database.nb_of_ports + 2 / data[0][1][0] * 2.32e-3 * _input_parts - cos(2*nb_of_days)'},
 'database': {'enabled': True,
  'nb_of_ports': 3,
  'ports': [8000, 8001, 8002],
  'data': [['delta', 'phi'], [3.14]],
  'temp_targets': {'cpu': 79.5, 'case': 72.0}},
 'servers': {'alpha': {'ip': '10.0.0.1', 'role': 'frontend'},
  'beta': {'ip': '10.0.0.2', 'role': 'backend'}}}

In [118]:
def find_formulas(dictionary: dict) -> dict[tuple, str]:
    formulas = {}
    for key, value in dictionary.items():
        if type(value) == str and value.startswith("="):
            formulas[(key,)] = value
        elif type(value) == dict:
            nested_dict = find_formulas(value)
            for nested_key, nested_value in nested_dict.items():
                formulas[(key, *nested_key)] = nested_value
        # elif type(value) in (list, tuple):
        #     for list_key, list_value in enumerate(value):

        #         nested_dict = find_formulas(list_value)
        #         for nested_key, nested_value in nested_dict.items():
        #             formulas[(list_key, *nested_key)] = nested_value
    return formulas


In [119]:
find_formulas(test_content)

{('owner',
  'calculated_value'): '= nb_of_days * database.nb_of_ports + 2 / data[0][1][0] * 2.32e-3 * _input_parts - cos(2*nb_of_days)'}

In [120]:
import pyparsing as pp

In [189]:
def decimal_parser() -> pp.ParseString:
    def convert_to_decimal(token):
        return Decimal(token[0])
    int_part = pp.Opt(pp.one_of('- +'))+pp.Word(pp.nums)
    mantissa = int_part + pp.Opt('.' + pp.Opt(pp.Word(pp.nums)))
    exponent_part = pp.Opt(pp.one_of('e E') + pp.Opt(pp.one_of('- +'))+ pp.Word(pp.nums))
    number = pp.Combine(mantissa + exponent_part)
    number.set_parse_action(convert_to_decimal)
    return number

def regular_number_parser():
    return pp.common.number

def var_name_parser():
    dict_like_key = pp.Suppress('.') + pp.common.identifier
    list_like_index = pp.Suppress('[')+pp.common.integer+pp.Suppress(']')
    subvar_element = dict_like_key ^ list_like_index
    return pp.Group(pp.common.identifier + subvar_element[...], aslist=True)

def function_struct_parser(number_parser):
    number = number_parser()
    return pp.common.identifier + pp.Suppress('(') + pp.Word(pp.alphanum) + pp.Suppress(')')

def build_operand_parser(number_parser):
    number = number_parser()
    var_name = var_name_parser()
    function_struct = function_struct_parser()

    operand = function_struct | number | var_name
    operator = pp.one_of("+ - * /")
    # operator_operand = operator + operand
    operator_operand = pp.infix_notation(integer | varname,
    [
        ('-', 1, pp.OpAssoc.RIGHT),
        (one_of('* /'), 2, pp.OpAssoc.LEFT),
        (one_of('+ -'), 2, pp.OpAssoc.LEFT),
    ])
    return pp.Suppress("=") + operator_operand

operand_parser = build_operand_parser(number_parser=regular_number_parser)

In [190]:
operand_parser.parse_string("=-2.233e3--2")

ParseResults([-2233.0, '-', -2], {})

In [191]:
operand_parser.parse_string(find_formulas(test_content)[('owner', 'calculated_value')])

ParseResults([['nb_of_days'], '*', ['database', 'nb_of_ports'], '+', 2, '/', ['data', 0, 1, 0], '*', 0.00232, '*', ['_input_parts'], '-', ['cos']], {})

In [192]:
type(operand_parser)

pyparsing.core.And

to be done :
- function handling : you need first to separate the contents of parentheses (function calls)
- calculation priority?